# Import Statements

In [ ]:
#Import dataset from Kaggle to Google Colab

In [19]:
#Create a file with name kaggle
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [28]:
#install kaggle
! pip install kaggle

In [29]:
! mkdir ~/.kaggle

mkdir: cannot create directory ‘/root/.kaggle’: File exists


In [ ]:
from google.colab import drive
files.upload()

In [30]:
!cp /content/kaggle.json ~/.kaggle/kaggle.json

In [31]:
! kaggle datasets download sachinkumarsaxena/male-female-face-image-training-testing-validation

100% 246M/246M [00:02<00:00, 105MB/s] 
100% 246M/246M [00:02<00:00, 86.9MB/s]


In [32]:
! mkdir train

mkdir: cannot create directory ‘train’: File exists


In [35]:
! unzip male-female-face-image-training-testing-validation.zip

Streaming output truncated to the last 5000 lines.
  inflating: Male_Female_Train_Test_Val/Validation/female/123257.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123261.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123263.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123266.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123267.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123269.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123270.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123275.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123280.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123286.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123292.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validation/female/123297.jpg.jpg  
  inflating: Male_Female_Train_Test_Val/Validatio

In [37]:
import os
import keras 
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from PIL import Image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
plt.style.use('dark_background')
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder 

# One Hot Encoding the Target Classes

In [38]:
encoder = OneHotEncoder()
encoder.fit([[0], [1]]) 

# 0 - female
# 1 - male

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

# Creating 3 Important Lists --
1. data list for storing image data in numpy array form
2. paths list for storing paths of all images
3. result list for storing one hot encoded form of target class whether male or female


In [39]:
# This cell updates result list for images with tumor

data = []
paths = []
result = []

for r, d, f in os.walk(r'/content/Male_Female_Train_Test_Val/Training/male'):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[0]]).toarray())

In [40]:
# This cell updates result list for images without tumor

paths = []
for r, d, f in os.walk(r"/content/Male_Female_Train_Test_Val/Training/female"):
    for file in f:
        if '.jpg' in file:
            paths.append(os.path.join(r, file))

for path in paths:
    img = Image.open(path)
    img = img.resize((128,128))
    img = np.array(img)
    if(img.shape == (128,128,3)):
        data.append(np.array(img))
        result.append(encoder.transform([[1]]).toarray())

In [41]:
data = np.array(data)
data.shape

(40882, 128, 128, 3)

In [43]:
result = np.array(result)
result = result.reshape(40882,2)

# Splitting the Data into Training & Testing

In [44]:
x_train,x_test,y_train,y_test = train_test_split(data, result, test_size=0.2, shuffle=True, random_state=0)

# Model Building

Batch normalization is a technique for training very deep neural networks that standardizes the inputs to a layer for each mini-batch. This has the effect of stabilizing the learning process and dramatically reducing the number of training epochs required to train deep networks.

In [45]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(2, 2), input_shape=(128, 128, 3), padding = 'Same'))
model.add(Conv2D(32, kernel_size=(2, 2),  activation ='relu', padding = 'Same'))


model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))
model.add(Conv2D(64, kernel_size = (2,2), activation ='relu', padding = 'Same'))

model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())

model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

model.compile(loss = "categorical_crossentropy", optimizer='Adamax')
print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 128, 128, 32)      416       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 128, 128, 32)      4128      
_________________________________________________________________
batch_normalization (BatchNo (None, 128, 128, 32)      128       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 64, 64, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 64, 64, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 64, 64, 64)        8256      
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 64, 64, 64)        1

In [46]:
y_train.shape

(32705, 2)

In [ ]:
history = model.fit(x_train, y_train, epochs = 3, batch_size = 40, verbose = 1,validation_data = (x_test, y_test))

Epoch 1/3
608/818 [=====================>........] - ETA: 8:02 - loss: 0.2160

# Plotting Losses

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Test', 'Validation'], loc='upper right')
plt.show()

# Just Checking the Model

In [ ]:
def names(number):
    if number==0:
        return 'Its a male'
    else:
        return 'Its a female'

In [ ]:
#check male 
from matplotlib.pyplot import imshow
img = Image.open(r"/content/Male_Female_Train_Test_Val/Validation/male/063429.jpg.jpg")
x = np.array(img.resize((128,128)))
x = x.reshape(1,128,128,3)
res = model.predict_on_batch(x)
classification = np.where(res == np.amax(res))[1][0]
imshow(img)
print(str(res[0][classification]*100) + '% Confidence This Is ' + names(classification))

In [ ]:
#check male
from matplotlib.pyplot import imshow
img = Image.open(r"../input/kidneydataset/KidneyDataset/Testing/young/IMG-0004-00011.jpg")
x = np.array(img.resize((128,128)))
x = x.reshape(1,128,128,3)
res = model.predict_on_batch(x)
classification = np.where(res == np.amax(res))[1][0]
imshow(img)
print(str(res[0][classification]*100) + '% Confidence This Is A ' + names(classification))

In [ ]:
#check female
from matplotlib.pyplot import imshow
img = Image.open(r"/content/Male_Female_Train_Test_Val/Testing/female/131422.jpg.jpg")
x = np.array(img.resize((128,128)))
x = x.reshape(1,128,128,3)
res = model.predict_on_batch(x)
classification = np.where(res == np.amax(res))[1][0]
imshow(img)
print(str(res[0][classification]*100) + '% Confidence This Is A ' + names(classification))

In [ ]:
#check female
from matplotlib.pyplot import imshow
img = Image.open(r"/content/Male_Female_Train_Test_Val/Testing/female/131423.jpg.jpg")
x = np.array(img.resize((128,128)))
x = x.reshape(1,128,128,3)
res = model.predict_on_batch(x)
classification = np.where(res == np.amax(res))[1][0]
imshow(img)
print(str(res[0][classification]*100) + '% Confidence This Is A ' + names(classification))

# Thats It !!